<a href="https://colab.research.google.com/github/linyuehzzz/hedetniemi_distance/blob/master/all_pair_distance_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**CUDA for all-pair distance algorithms**
CUDA parallelism for all-pair distance algorithms.  
Yue Lin (lin.3326 at osu.edu)  
Created: 6/12/2020

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

#### **Install packages** 

In [0]:
!pip install timeout-decorator

#### **CUDA device query** 

In [1]:
!nvcc --version
from numba import cuda
print(cuda.gpus)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
<Managed Device 0>


In [7]:
%cd /usr/local/cuda-10.1/samples/1_Utilities/deviceQuery
!ls

/usr/local/cuda-10.1/samples/1_Utilities/deviceQuery
deviceQuery.cpp  Makefile  NsightEclipse.xml  readme.txt


In [9]:
!make
!./deviceQuery

make: Nothing to be done for 'all'.
./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla P100-PCIE-16GB"
  CUDA Driver Version / Runtime Version          10.1 / 10.1
  CUDA Capability Major/Minor version number:    6.0
  Total amount of global memory:                 16281 MBytes (17071734784 bytes)
  (56) Multiprocessors, ( 64) CUDA Cores/MP:     3584 CUDA Cores
  GPU Max Clock rate:                            1329 MHz (1.33 GHz)
  Memory Clock rate:                             715 Mhz
  Memory Bus Width:                              4096-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory: 

#### **Read graph data** 

In [0]:
%cd '/content/gdrive/My Drive/Colab Notebooks/hedetniemi_matrix_sum'

## Number of nodes (100/1,000/10,000/100,000/1,000,000)
nodes = 100
print('Nodes: ', nodes)
## Total degree
degree = 3
print('Degree: ', degree)

data = []
with open('graph_n' + str(nodes) + '_d' + str(degree) + '.txt', 'r') as f:
  lines = f.read().splitlines()
  for line in lines:
    l = line.split()
    item = [int(l[0]), int(l[1]), float(l[2])]
    data.append(item)

print(data[0])

#### **Hedetniemi distance** 

##### Construct distance matrix

In [0]:
from timeit import default_timer
from numba import cuda
import numpy as np
import timeout_decorator

@timeout_decorator.timeout(10)
@cuda.jit
def distance_matrix(graph):
  ## calculate number of nodes
  n = int(np.amax(graph[:,1]))

  ## calculate distance matrix
  dist_mtx = np.full((n,n), np.inf)
  for g in range(graph.shape[0]):
    i = int(graph[g][0]) - 1
    j = int(graph[g][1]) - 1
    d = graph[g][2]
    dist_mtx[i,j] = d
    dist_mtx[j,i] = d

  ## set diagonal to 0
  np.fill_diagonal(dist_mtx, 0)
 
  return dist_mtx, n


## print time costs
try:
  start = default_timer()
  dist_mtx, n = distance_matrix(np.array(data))
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')

## print distance matrix
# print("Distance matrix: ")
# for line in dist_mtx:
#   print(line)

##### Calculate Hedetniemi Matrix Sum

In [0]:
from timeit import default_timer
from numba import cuda
import numpy as np
import timeout_decorator

@timeout_decorator.timeout(10)
@cuda.jit
def hede_distance(matrix, n):
  mtx_a_t = np.full((n,n), np.inf)
  mtx_a_t_1 = matrix

  p = True
  while p:
    for i in range(n):
      a = mtx_a_t_1[i]
      for j in range(n):
        b = matrix[:,j]
        mtx_a_t[i,j] = np.amin(np.array([a[k] + b[k] for k in range(n)]))
    
    if np.array_equal(mtx_a_t, mtx_a_t_1):
      p =  False
    else:
      mtx_a_t_1 = mtx_a_t   
  
  return mtx_a_t


## print time costs
try:
  start = default_timer()
  mtx_a_t = hede_distance(dist_mtx, n)
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')

## print shortest path matrix
with open('hedet_mtx_nb_cuda.txt', 'w') as fw:
  fw.write('\n'.join(['\t'.join([str(cell) for cell in row]) for row in mtx_a_t]))

Time:  0.6015514100008659
